CisReg datbase:

* Cis_include_genome2  or Cis_include_mRNA
  * RFXXXX/
    * RFFXXXX.filtered.clustal
    * RFFXXXX.filtered.struct
    * Cis_flanks-XX/
      * ID-SS-EE_known_nt.fasta


Overall Procedure:
For each family:
    1. Read sequences and alignment from RFFXXXX.filtered.clustal
    2. For each pair of sequence compute PSI and SCI. For the PSI, sequences are reliagned but for the SCI Rfam alignmnet is passed to RNAalifold
    3. From pairs with PSI>0.95 only keep one. This is to avoid having too similar alignments at the end.
    3. Write the pair as a fasta ref file
    3. Find the two contextadded fasta files of the pair from *known_nt.fasta
      . If not found swap start end and search for that file
    4. Merge the known_nt.fasta files and report as raw file
    
** Note: In LocalFold dataset seq ids on reverse strand have Rfam-id with swapped start and end locations**

In [1]:
import notebook
E = notebook.nbextensions.EnableNBExtensionApp()
E.enable_nbextension('usability/codefolding/main')

import glob
import os, sys
from Bio import AlignIO, SeqIO
from Bio.Align import MultipleSeqAlignment
import itertools



# settings !!!! IMPORTANT UPDATE IT to CisDataset folder !!!!!!
EXTREME_SI = 95 # from two extermly similar seqs discard one
db_path = '/home/milad/DataBase/CisReg/Cis_include_genome2/'
regx_gaps = '[-.~_]'  # valid gap symbols all be converted to "-"

VIENNA_BIN_PATH = '/home/milad/software/bin/'
RNAFOLD =  'RNAfold -p --noPS ' 
RNAPLFOLD ='RNAplfold '
import numpy as np

# Import libraries located in relation to this file
# tools_dir = '/home/milad/workspace/rnaalignclust/bin/analysis/tools/' #os.path.join(parent_dir, '/bin/analysis/tools/')
# print "tools_dir", tools_dir
# sys.path.insert(0, tools_dir)


import re

# -------------
def remove_gap_columns(malign):
    ''' Remove all-gap columns from a biopython multiple-alignment
    Returns pruned multiple-alignment'''

    if (len(malign) == 0):
        return
    for c in reversed(range(len(malign[0]))):
        if len(re.sub(regx_gaps, '', malign[:, c])) == 0:
            malign = malign[:, 0:c] + malign[:, c + 1:]  # concat left & right side of column c
    
    return malign

   # End def remove_gap_columns   


In [3]:
from subprocess import *

def sub_dotplot(dp, lcontext, rcontext):
    '''Returns an squared submatrix by removing pre and post section ....
    the aim is to extract dotplot of RNA from a dotplot of the context extened RNA'''
    assert (dp.shape[0] == dp.shape[1])
    assert (dp.shape[0] >= lcontext + rcontext)
    n = dp.shape[0]
    return dp[lcontext:n-rcontext, lcontext:n-rcontext]

def upper_part (dpX):
    ''' Returns the upper section of matrix in flattened form, expecting to have bp probabilities'''
    
    return dpX[np.triu_indices(dpX.shape[0],1)]

# TODO: Add all repeatedly used functions into a library accros all tools

def dotbracket_to_dict(struct):
    '''Returns a dictionary where basepairs are keys with !ONE! based indices joined by ":" ,
    e.g. dict {'0:10': 1, '2:8': 1} '''
    assert len(struct.replace('.', '').replace('(', '').replace(')', '')) == 0
    stack = list()
    pairs = dict()
    for pos, ch in enumerate(list(struct)):
        #         print pos+1, ch
        if ch=='(':
            stack.append(pos)
        elif ch==')':
            left = stack.pop()
            key= "{}:{}".format(left+1, pos+1)
            pairs[key] = 1
    
    assert len(stack) == 0
    return pairs


def compute_part_func(infile_fa, seq_names, outdir_path="./", use_plfold=False, use_cache = False):
    '''Runs Vienna RNAfold/RNAplfold with partition function for all sequences inside input fasta file
    If use_cache, it does nothing if If the ps file with same paramaters exists '''
    from subprocess import Popen, PIPE
    #     print "compute_part_func(", infile_fa, seq_names
    if use_plfold:
        out_dir = outdir_path + RNAPLFOLD.replace(' ', '')
    else:
        out_dir = outdir_path + RNAFOLD.replace(' ', '')
    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)
        
    all_in_cache = all([os.path.isfile(os.path.join(out_dir, sname+'_dp.ps')) for sname in seq_names])
    if all_in_cache and use_cache:
        return out_dir
    
    with open(infile_fa) as in_rna:
        
        if use_plfold:
            p = Popen(('cd %s;' %out_dir) + VIENNA_BIN_PATH + RNAPLFOLD , stdin=in_rna, shell=True, stdout=PIPE, stderr=PIPE)
        else:
            p = Popen(('cd %s;' %out_dir) + VIENNA_BIN_PATH +  RNAFOLD , stdin=in_rna, shell=True, stdout=PIPE, stderr=PIPE)
        
        out, err = p.communicate()
        if err:
            print "Error in calling RNAfold for ", infile_fa
            print out
            print err
            if not use_plfold:
                raise RuntimeError
    
    return out_dir

def parse_dp_ps(ps_file):
    '''Extracts base pair probabliies from vienna ps file
    returns: Dictinary of form dict[i:j]=p(i,j) '''
    
    # Extract sequence from ps file
    myseq = ""
    read_seq = False
    with open(ps_file) as in_ps:
        for line in in_ps:
            if "/sequence" in line:
                read_seq = True
            elif read_seq and ") } def" in line:
                read_seq = False
            elif read_seq:
                myseq += line.rstrip().rstrip("\\")
    #     print ps_file.rstrip("_dp.ps") , myseq
              
    import re
    ureg = re.compile(r'^(\d+)\s+(\d+)\s+(\d+\.\d+)\s+ubox\s*')
    bp_prob_dict = dict()
    bp_prob_mat = np.zeros((len(myseq), len(myseq)))
    
    with open(ps_file) as in_ps:
        for line in in_ps:
            if "ubox" in line:
                um = ureg.match(line)
                if um:
                    i, j, sqrp = um.groups()

                    #                     print i, j, sqrp
                    
                    # keys are pair of indexes as smaller:larger
                    key = ":".join(sorted([i,j], reverse=True))
                    assert (key not in bp_prob_dict)
                    bpprob = float(sqrp)*float(sqrp)
                    bp_prob_dict[key] = bpprob
                    
                    i,j = int(i), int(j)
                    bp_prob_mat[i-1,j-1] = bpprob             
    return bp_prob_mat

In [15]:
def decode_cisreg_entry(famid, flank_len):
    ''' Input is a RFAM id and the flanking size
    Returns Constrained folded(reference) structure and the clustal alignment in format
    [AlignIO, pandas_df_struct]
    '''
    
    # ============================= A =================================
    # set file and dirs names then check exitance and remove files within outdir
    clust_filtered_file = '{}{}/{}.filtered.clustal'.format(db_path, famid, famid )
    struct_file = '{}{}/{}.struct'.format(db_path, famid, famid )
    fam_flank_path = '{}{}/Cis_flanks-{}/'.format(db_path, famid, flank_len )
    dp_out_path = "./dp-ps-Cis-flanks-{}/".format(flank_len)
    
    if not os.path.isfile(clust_filtered_file):
        raise IOError("Clustal file not found: {}".format(clust_filtered_file))
    if not os.path.isfile(struct_file):
        raise IOError("CisReg-Struct file not found: {}".format(clust_filtered_file))
    if not os.path.isdir(fam_flank_path):
        raise IOError("Family flanking dir does not exist: {}".format(fam_flank_path))
    
    if not os.path.isdir(dp_out_path):
        os.mkdir(dp_out_path)

    # ============================= B =================================
    # Read struct file
    import pandas as pd
    df_struct = pd.DataFrame.from_csv(struct_file, sep="\t")
    print "Number of sequences in .struct: ", len(df_struct)
    
    df_struct['flanked-id'] = 'flanked'

    # ============================= C =================================
    # Read clustal file

    clustal_handle = open(clust_filtered_file, 'r')

    clustal_alignment = AlignIO.read(clustal_handle, "clustal")
    print "Number of sequences in alignment: ", len(clustal_alignment)
    
    # ============================= C =================================
    # Sanity check struct and clustal sequnces match
    if len(df_struct) !=  len(clustal_alignment):
#         raise RuntimeError
        print ("WARNING: decode_cisreg_entry({}) len(df_struct) !=  len(clustal_alignment) {}!={} \n".format(
                           famid, len(df_struct), len(clustal_alignment)))
    
    assert len(df_struct) <=  len(clustal_alignment) # TODO: Why some sequnces are missing from the .struct ?
    for seq in clustal_alignment:
        print seq.id,
        seq_reverese_corrected = seq.id
        if seq.id not in df_struct.index:
            # On reverse strand the starting ending positions are swapped, so check for both
            splits = seq.id.replace("/", " ").replace("-", " ").split()
            assert(len(splits)==3)
            seq_id_reverse = "{}/{}-{}".format(splits[0], splits[2], splits[1])
            if seq_id_reverse not in df_struct.index: # TODO: Why some sequnces are missing from the .struct ?
#                 raise RuntimeError
                print(" WARNING decode_cisreg_entry Fam:{} struct-clustal mismatch for seq {}\n".format( 
                       famid, seq.id))
            seq_reverese_corrected = seq_id_reverse
            

        # ============================= D =================================
        # Get the fasta file of specific flanking range

        fasta_flanked_seq = "{}/{}_known_nt.fasta".format(fam_flank_path, seq.id.replace("/", "_") )
        # Sometimes the starting ending positions are swapped, redfine fasta var
        if not os.path.isfile(fasta_flanked_seq):
            splits = seq.id.replace("/", " ").replace("-", " ").split()
            assert(len(splits)==3)
            
            fasta_flanked_seq = "{}/{}_known_nt.fasta".format(fam_flank_path, "{}_{}-{}".format(splits[0], splits[2], splits[1]) )
            if not os.path.isfile(fasta_flanked_seq):
                raise IOError("Fasta file not found: {}".format(fasta_flanked_seq))
#             print "reverse strand"
        
        # Get the extended_id, which is different from seq.id when flanking is non-zero
        with open(fasta_flanked_seq, "r") as in_fasta_handle:
            fa_recs = list(SeqIO.parse(in_fasta_handle, "fasta"))
        assert len(fa_recs) == 1
        fasta_flanked_id =  fa_recs[0].id
        print fasta_flanked_id
        df_struct.set_value(seq_reverese_corrected, 'flanked-id', fasta_flanked_id)
        
        
        dp_outdir = compute_part_func(fasta_flanked_seq, [seq.id], outdir_path=dp_out_path, use_plfold=False)
        
    return clustal_alignment, df_struct, dp_outdir





def get_expected_accuracy(reference_struct, dp_matrix):
    '''dp_matrix is a numpy matrix where base indeices are ZERO based'''
    print len(reference_struct), dp_matrix.shape
    assert dp_matrix.shape[0] == dp_matrix.shape[1]
    assert dp_matrix.shape[0] == len(reference_struct)
    reference_struct_dict = dotbracket_to_dict(reference_struct)
    sum_TP_prob = 0.0
    for bp_key in reference_struct_dict:
        i,j = bp_key.split(":")
        i,j = int(i), int(j)
        sum_TP_prob += dp_matrix[i-1,j-1]
#         print i,j, dp_matrix[i-1,j-1]
    
    print "    TP_score: %.2f" % (sum_TP_prob/len(reference_struct_dict))
    return sum_TP_prob

# Run the tool per family and per context length
context_len = 100
currentfam = 'RF00140'
fam_alignment, df_fam_context, dp_dir = decode_cisreg_entry(currentfam, context_len)
# for seq in list(fam_alignment)[0:1]:
for seq_id in list(df_fam_context.index)[0:]:
    extended_seq_id = df_fam_context['flanked-id'][seq_id]
    extended_seq_id = extended_seq_id.replace('/','_')
    
    dp_ps = './dp-ps-Cis-flanks-{}/RNAfold-p--noPS/{}_dp.ps'.format(context_len, 
                                                                    extended_seq_id)
    print seq_id, extended_seq_id, dp_ps
    assert(os.path.isfile(dp_ps))
    dp_matrix = parse_dp_ps(dp_ps)
    
    get_expected_accuracy(df_fam_context['STRUCTURE_CONSTRAINT_MFE'][seq_id], dp_matrix)
#     get_expected_accuracy(df_fam_context['STRUCTURE_CONSTRAINT'][seq_id], parse_dp_ps(dp_ps))
fam_alignment

Number of sequences in .struct:  39
Number of sequences in alignment:  39
CR378663.1/343377-343482 CR378663.1_343277-343582
CP000388.1/1197645-1197749 CP000388.1_1197545-1197849
BX571874.1/272618-272731 BX571874.1_272831-272518
CP000753.1/256694-256792 CP000753.1_256594-256892
CP000503.1/201038-201135 CP000503.1_200938-201235
AADP01000001.1/262333-262447 AADP01000001.1_262233-262547
AE016853.1/702273-702368 AE016853.1_702173-702468
AE016827.1/1995612-1995725 AE016827.1_1995825-1995512
CP000510.1/4332391-4332491 CP000510.1_4332591-4332291
BX640437.1/48199-48307 BX640437.1_48099-48407
AAPS01000075.1/7302-7409 AAPS01000075.1_7202-7509
CR954246.1/2987758-2987868 CR954246.1_2987968-2987658
AAMX01000030.1/15346-15445 AAMX01000030.1_15246-15545
BX950851.1/4490505-4490615 BX950851.1_4490715-4490405
BX936398.1/4386675-4386785 BX936398.1_4386885-4386575
AC202195.2/54062-54176 AC202195.2_54276-53962
ABCH01000098.1/5690-5799 ABCH01000098.1_5590-5899
CP000746.1/527966-528079 CP000746.1_527866-52817

AssertionError: 

In [33]:
list(fam_alignment)

[SeqRecord(seq=Seq('GGGGCUGAUUCAGGAUU.CGACAGGA.UCAAGAAG...GCUUG.UGGA....GC...ACC', SingleLetterAlphabet()), id='CR378665.1/70600-70234', name='<unknown name>', description='CR378665.1/70600-70234', dbxrefs=[]),
 SeqRecord(seq=Seq('GGGGCUGAUUCUGGAUU.CGACGGGA.UUCGCGAA...ACCCA.AGGU....GC...ACC', SingleLetterAlphabet()), id='BX571870.1/186841-187203', name='<unknown name>', description='BX571870.1/186841-187203', dbxrefs=[]),
 SeqRecord(seq=Seq('GGGGGCGAAUAUGGUUU.CGACAUGA.AUGUCAAA...AUCUA.AGGU....GC...ACC', SingleLetterAlphabet()), id='AJ749949.1/1737842-1738261', name='<unknown name>', description='AJ749949.1/1737842-1738261', dbxrefs=[]),
 SeqRecord(seq=Seq('GGGGGCGAC.CUGGUUU.CGACAGGG.GUUGCGAA...GCGGC.UAGG....GC...ACC', SingleLetterAlphabet()), id='CP001043.1/829081-828713', name='<unknown name>', description='CP001043.1/829081-828713', dbxrefs=[]),
 SeqRecord(seq=Seq('GGGGCUGAUUCUGGAUU.CGACAAGA.UUCACGAA...ACCCA.AGGU....GC...ACC', SingleLetterAlphabet()), id='AY442269.2/10-368', name='<u

In [10]:
# print "\n".join(df_fam_context['flanked-id'])
df_fam_context

,RATIO_REMAINING_CONSTRAINT,STRUCTURE_CONSTRAINT,STRUCTURE_CONSTRAINT_MFE,RATIO_CONSTRAINT_MFE,ALL_ENSEMBLE_ENERGY,CONSTRAINT_ENSEMBLE_ENERGY,CONSTRAINT_MFE_ENERGY,CONSTRAINT_PROBABILITY,CONSTRAINT_MFE_PROBABILITY,REL_DIFF_CONSTRAINT_P_AND_CONSTRAINT_MFE_P,flanked-id
NAME,,,,,,,,,,,
BX936398.1/4386785-4386675,0.888889,.....((((((((....................................,..((.(((((((((((((((...((((....))))...))))).))...,1,-36.25,-36.25,-33.70,1.000000e+00,1.596333e-02,0.984037,BX936398.1_4386885-4386575
CP000510.1/4332491-4332391,1.000000,.....((((((((......................))))))))......,.....((((((((.(((..(((....)))..))).))))))))......,1,-23.17,-23.14,-20.90,9.524897e-01,2.514361e-02,0.973602,CP000510.1_4332591-4332291
AADP01000001.1/262333-262447,1.000000,...((((((((((....................................,...(((((((((((((((((...((((....))))...)))))).....,1,-35.45,-35.45,-33.20,1.000000e+00,2.597292e-02,0.974027,AADP01000001.1_262233-262547
AE017340.1/2033365-2033265,1.000000,...(((..(((((.......................))))).)))....,...(((..(((((((((..(((....)))...))))))))).)))....,1,-32.62,-32.62,-30.80,1.000000e+00,5.218233e-02,0.947818,AE017340.1_2033465-2033165
CP000753.1/256694-256792,1.000000,....(((((((((......................))))))))).....,....((..(((((.(((..(((....)))..))).)))))..)).....,0.777777777777778,-29.46,-28.52,-26.40,2.175815e-01,6.978277e-03,0.967928,CP000753.1_256594-256892
CP000507.1/262573-262670,1.000000,....(((((((((......................))))))))).....,....(((((((((.(((..(((....)))..))).))))))))).....,1,-32.23,-32.23,-30.10,1.000000e+00,3.155583e-02,0.968444,CP000507.1_262473-262770
CP000388.1/1197645-1197749,1.000000,...((.(((((((..............................)))...,......(((((((((..(((..(((....)))..)))....)))))...,0.777777777777778,-26.88,-26.76,-23.50,8.230782e-01,4.152004e-03,0.994956,CP000388.1_1197545-1197849
AE005174.2/4244784-4244673,1.000000,...((((((((((....................................,...((((((((((...(((((...((((....))))...))))).....,1,-37.72,-34.89,-32.60,1.013490e-02,2.466913e-04,0.975659,AE005174.2_4244884-4244573
CP000436.1/76674-76792,1.000000,...((((((((((....................................,...(((((((((((((((((...((((...))))...))))))..)...,1,-34.80,-34.80,-32.20,1.000000e+00,1.471942e-02,0.985281,CP000436.1_76574-76892


In [22]:

# Removed backup code:
    
    # ============================= D =================================
    # Go through the the pair of sequences again
    # Calculate SCI, ignore low SCI pairs and write the remaining ones into raw.fa files

    psi_occurances_dict = dict()

    for i, seq1 in enumerate(fam_alignment):
    #     print seq1.id
        if (seq1.id not in pruned_id_list):
#             print "not in list" + seq1.id
            continue
        for j in range(i+1, len(fam_alignment)):
            seq2 = fam_alignment[j]
            if (seq2.id not in pruned_id_list):
                continue

            # === Calculate SCI ====
            clust_str = "CLUSTAL\n{}    {}\n{}    {}\n@".format(seq1.id, seq1.seq, seq2.id, seq2.seq)
            outdata = Popen('/usr/bin/RNAalifold --sci ', stdin=PIPE, shell=True, stdout=PIPE, stderr=STDOUT).communicate(clust_str)
            lastline = ""
            for line in outdata:
                if line is not None and len(line) > 0: 
                    lastline = line
            match = re.search(r'\[(\d+\.\d+)\]', lastline)
            SCI = int(round(float(match.group(1))*100))
            if (SCI < 0.60):
                print "Low SCI ", SCI, key
            # === ------------- ====

            key  = ':'.join(sorted({seq1.id, seq2.id}))
    #         print psi_dict[key]
            uniqueness_key = '{}.apsi-{}'.format(famid, psi_dict[key])
            if uniqueness_key in psi_occurances_dict:
                psi_occurances_dict[uniqueness_key]+=1
            else:
                psi_occurances_dict[uniqueness_key] = 1

            
            
            basename = '{}.sci-{}.no-{}'.format(uniqueness_key, SCI, psi_occurances_dict[uniqueness_key]) # This no number is for all same APSIs and not correct
            
            # write ref file
            pair2file(seq1, seq2, psi_dict[key], output_path+basename+'.ref.fa', 'fasta')
    #         pair2file(seq1, seq2, psi_dict[key], output_path+basename+'.ref.clustal', 'clustal')

    #         print seq1.id, seq2.id

            ## -Write raw files (with flanking region)
            fasta1 = "{}/{}_known_nt.fasta".format(fam_flank_path, seq1.id.replace("/", "_") )
    #         fasta1 = blob.blob("{}/{}_known_nt.fasta".format(fam_flank_path, seq1.id.replace("/", "_") )
            
            # Sometimes the starting ending positions are swapped, so check for both
            if not os.path.isfile(fasta1):
                splits = seq1.id.replace("/", " ").replace("-", " ").split()
                assert(len(splits)==3)
                fasta1 = "{}/{}_known_nt.fasta".format(fam_flank_path, "{}_{}-{}".format(splits[0], splits[2], splits[1]) )
                if not os.path.isfile(fasta1):
                    raise IOError("Fasta file not found: {}".format(fasta1))


            fasta2 = "{}/{}_known_nt.fasta".format(fam_flank_path, seq2.id.replace("/", "_") )
            
            # Sometimes the starting ending positions are swapped, so check for both
            if not os.path.isfile(fasta2):
                splits = seq2.id.replace("/", " ").replace("-", " ").split()
                assert(len(splits)==3)
                fasta2 = "{}/{}_known_nt.fasta".format(fam_flank_path, "{}_{}-{}".format(splits[0], splits[2], splits[1]) )
                if not os.path.isfile(fasta2):
                    raise IOError("Fasta file not found: {}".format(fasta2))

            # write the raw file
            merge_fastas(fasta1, fasta2, output_path+basename+'.raw.fa')
        
# for pair in itertools.permutations(fam_alignment, 2):
#  print sequence_identity_realign( pair[0].seq,  pair[1].seq), pair[0].id, pair[1].id


IndentationError: unexpected indent (<ipython-input-22-36361981ee2b>, line 8)